In [1]:
from datetime import datetime
import pandas as pd

In [2]:
#import devanalyst.simulation.statics as S_

from devanalyst.simulation.simulationModels import ModelsConfig
#, DefaultCostModel, MeritocraticCostModel, \
#DistributedLagQualityModel, GreedyAllocationModel, NoLaggardsAllocationModel

from devanalyst.simulation.businessObjects import TicketsRepo, Backlog, UserStoryStatus

#from devanalyst.metrics.ProductivityMetrics import CommitsAnalysis
import devanalyst.simulation.generateTimecards as timecard

importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\simulationModels.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\generateTimecards.ipynb


In [4]:
class ReleaseSimulationEngine:
    
    # -modelsConfig: configuration for the release simulation, inclusive of all simulation models as well as the 
    #                data repositories that should be pre-populated with relevent user stories to implement in this
    #                release cycle when the simulation is run, as well as the team that is to implemen them.
    def __init__(self, modelsConfig):
        self.modelsConfig = modelsConfig
        
        # Objects computed in the process of running the engine
        self.entries_df = None
        self.log        = None
        
    # Simulates the release process from the given start_date and the given number_of_sprints. Sets the
    # 'self.entries_df' and 'self.log' over this period of time.
    #
    # -number_of_sprints: integer corresponding to how many sprints we should simulate for the release cycle. Normally
    #                     it would be set to be high enough so that all user stories are completed over those sprints.
    # -sprint_duration: integer corresponding to the duration of a typical sprint, in days.
    # -start_date: a datetime instance corresponding to the date when the release simulation starts. Therefore, all
    #              timecard entries are dated starting with the end date of the first sprint that begins on this start_date.
    def run(self, number_of_sprints, sprint_duration, start_date):
        entries_df, log = timecard.runReleaseCycle(start_date, 
                                                   sprint_duration, 
                                                   number_of_sprints,
                                                   self.modelsConfig)
        self.entries_df = entries_df
        self.log        = log
       
    # Resets the release simulation engine by bringing it back to the state it was in right before it was run.
    # Thus, the teams and original user stories are the same, but all progress done on user stories (as tracked in
    # the teams' backlogs' UserStoryStatus objects) are cleared out. Also all tickets are deleted.
    # Thus, it leaves things in a state where the release simulation engine can be run again, after possibly some
    # changes (such as maybe changing the team or some algorithmic parameter in modelsConfig). This allows establishing
    # comparison on release cycle outcomes when the same user story workload is implemented under different assumptions.
    def resetEngine(self):
        teamsRepo = self.modelsConfig.globalRepo.teamsRepo
        for team in teamsRepo.teams:
            # Now clear the user story statuses for each story in a team's backlog
            old_pendingStories = team.backlog.pendingUserStories
            team.backlog = Backlog()
            for old_uss in old_pendingStories:
                storyId = old_uss.userStoryId
                team.backlog.pendingUserStories.append(UserStoryStatus(storyId)) 
        self.modelsConfig.globalRepo.ticketsRepo = TicketsRepo()